In [ ]:
# System prompts, multi-shot prompting, and RAG

In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from IPython.display import display, Markdown

In [ ]:
# Load environment variables in a file .env

load_dotenv()
openai_key=os.getenv("OPENAI_API_KEY")

# Check the key
if not openai_key:
    print("An API key was not found")
elif openai_key[:8]!="sk-proj-":
    print("An API key was found, but it dosen't start with sk-proj-")
elif openai_key.strip()!= openai_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them")
else:
    print("An API key was found and looks good")

In [ ]:
# Check if Ollama is running by making a proper request
try:
    response = requests.get("http://localhost:11434")
    print(f"Status Code: {response.status_code}")
    print(f"Response: {response.text}")
except requests.exceptions.ConnectionError:
    print("Cannot connect to Ollama. Make sure it's running on localhost:11434")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Ollama parameters
OLLAMA_BASE_URL  = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

olama_model = "llama3.2"

In [ ]:
# Create an instance of openai
openai = OpenAI()
openai_model = "gpt-4o-mini"

In [ ]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."


In [ ]:
# system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
# but remind the customer to look at hats!"

In [ ]:
def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=openai_model, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
# UI implementation with gradio
gr.ChatInterface(fn=chat, title="Xamagos").launch(inbrowser=True)